In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

d:\Documents\SelfProject\IndoRecipe2


In [2]:
import pandas as pd

In [3]:
from idrecibrew2.data import Seq2SeqDataFactory, Seq2SeqDataFactoryArgs
from idrecibrew2.data.indonlg_tokenizer.tokenizer import IndoNLGTokenizer

C:\Users\62852\miniconda3\envs\idrecibrew2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = IndoNLGTokenizer.from_pretrained("indobenchmark/indobart-v2")
data_args = Seq2SeqDataFactoryArgs(
    source_column="src",
    label_column="tgt",
    tokenizer=tokenizer,
)
data_factory = Seq2SeqDataFactory(data_args=data_args)
dtl = data_factory.produce_dataloader_from_csv(
    csv_file="tests/data_dummy/test.csv", batch_size=3, n_workers=1, shuffle=True
)

100%|██████████| 1/1 [00:00<00:00, 333.60ba/s]


In [5]:
from pytorch_lightning import Trainer

In [6]:
trainer = Trainer(fast_dev_run=True)

C:\Users\62852\miniconda3\envs\idrecibrew2\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_test_batches=1)` was configured so 1 batch will be used.
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [10]:
from idrecibrew2.model import LitSeq2SeqTransformers, LitSeq2SeqTransformersArgs
from idrecibrew2.eval import Seq2SeqTrainingEval
from idrecibrew2.data.indonlg_tokenizer.tokenizer import IndoNLGTokenizer


In [14]:
eval_obj = Seq2SeqTrainingEval(tokenizer=tokenizer)
lit_model_args = LitSeq2SeqTransformersArgs(
    model_type="indobart-v2",
    vocab_size=tokenizer.vocab_size,
    optimizer_type="adam",
    learning_rate=1e-5,
)
lit_model = LitSeq2SeqTransformers(config=lit_model_args)
lit_model.set_eval_object(eval_obj)

In [15]:
trainer.fit(lit_model, dtl, dtl)


  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | MBartForConditionalGeneration | 131 M 
--------------------------------------------------------
131 M     Trainable params
0         Non-trainable params
131 M     Total params
526.172   Total estimated model params size (MB)
C:\Users\62852\miniconda3\envs\idrecibrew2\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\62852\miniconda3\envs\idrecibrew2\lib\site-packages\pytorch_lightning\trainer\trainer.py:1938: PossibleUserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower va

Epoch 0: 100%|██████████| 2/2 [00:07<00:00,  3.79s/it, loss=8.91, v_num=]

TypeError: list indices must be integers or slices, not str